In [1]:
import torch
from transformers import AutoProcessor
from transformers import SeamlessM4TModel

processor = AutoProcessor.from_pretrained("facebook/hf-seamless-m4t-Large", use_fast=False, force_download=True)
model = SeamlessM4TModel.from_pretrained("facebook/hf-seamless-m4t-Large")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/ubuntu/miniconda3/envs/mmmc/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/ubuntu/miniconda3/envs/mmmc/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/miniconda3/envs/mmmc/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/miniconda3/envs/mmmc/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, 

In [2]:
import jieba
src = "Inspired by the success in NLP, embodied agent research [29,11,94,23] has seen a surge in adoption of the large-scale pre-training paradigm. The recent advances can be roughly divided into 4 categories. 1) Novel agent architecture: Decision Transformer [19,58,144] applies the powerful self-attention models to sequential decision making."
tgt = "受到NLP的成功启发,体现代理研究 [29,11,94,23] 已经看到大规模预训练范式的采用激增. 最近的进步可以大致分为4个类别. 1) 新型代理架构:决策变换器 [19,58,144] 应用强大的自我注意模型来进行顺序决策."

seamless_lang_dict = {
    "Arabic": "arb",
    "Chinese": "cmn",
    "English": "eng",
    "French": "fra",
    "Japanese": "jpn",
    "Russian": "rus",
}

In [3]:
import itertools
sent_src, sent_tgt = src.split(), jieba.cut(tgt, cut_all=False)
token_src, token_tgt = [processor.tokenizer.tokenize(word) for word in sent_src], [processor.tokenizer.tokenize(word) for word in sent_tgt]
wid_src, wid_tgt = [processor.tokenizer.convert_tokens_to_ids(x) for x in token_src], [processor.tokenizer.convert_tokens_to_ids(x) for x in token_tgt]
ids_src, ids_tgt = processor.tokenizer.prepare_for_model(list(itertools.chain(*wid_src)), return_tensors='pt', model_max_length=processor.tokenizer.model_max_length, truncation=True)['input_ids'], processor.tokenizer.prepare_for_model(list(itertools.chain(*wid_tgt)), return_tensors='pt', truncation=True, model_max_length=processor.tokenizer.model_max_length)['input_ids']
sub2word_map_src = []
for i, word_list in enumerate(token_src):
  sub2word_map_src += [i for x in word_list]
sub2word_map_tgt = []
for i, word_list in enumerate(token_tgt):
  sub2word_map_tgt += [i for x in word_list]

Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens':

In [25]:
# alignment
align_layer = 8
threshold = 1e-3
model.eval()
with torch.no_grad():
  out_src = model(ids_src.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]
  out_tgt = model(ids_tgt.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]

  dot_prod = torch.matmul(out_src, out_tgt.transpose(-1, -2))

  softmax_srctgt = torch.nn.Softmax(dim=-1)(dot_prod)
  softmax_tgtsrc = torch.nn.Softmax(dim=-2)(dot_prod)

  softmax_inter = (softmax_srctgt > threshold)*(softmax_tgtsrc > threshold)

align_subwords = torch.nonzero(softmax_inter, as_tuple=False)
align_words = set()
for i, j in align_subwords:
  align_words.add( (sub2word_map_src[i], sub2word_map_tgt[j]) )

This calls the same method `forward` as `SeamlessM4TForTextToText` and `SeamlessM4TForSpeechToText`depending on the input modality. If you want to generate speech, use the `generate` method.


ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds

In [35]:
from transformers import SeamlessM4TPreTrainedModel

text_decoder_input_ids = None
tgt_lang=seamless_lang_dict['Chinese']
# overwrite text_decoder_input_ids if tgt_lang is passed. The latter gets priority over decoder_input_ids.
if tgt_lang is not None:
    # tgt_lang gets priority over decoder input ids
    text_tgt_lang_id = model.generation_config.text_decoder_lang_to_code_id.get(tgt_lang)
    text_decoder_input_ids = torch.tensor([[text_tgt_lang_id]] * 1).to(model.device)
tmp = model(input_ids = ids_src.unsqueeze(0), decoder_input_ids = text_decoder_input_ids, output_hidden_states=True)

This calls the same method `forward` as `SeamlessM4TForTextToText` and `SeamlessM4TForSpeechToText`depending on the input modality. If you want to generate speech, use the `generate` method.


In [41]:
tmp.__dict__.keys()

dict_keys(['loss', 'logits', 'past_key_values', 'decoder_hidden_states', 'decoder_attentions', 'cross_attentions', 'encoder_last_hidden_state', 'encoder_hidden_states', 'encoder_attentions'])

In [37]:
tmp[2]
# difference: this is an encoder-decoder model; graham's is an encoder-only model

(tensor([[[  4.9665,   0.0387,  -1.3188,  ..., -17.2500,  12.2500,  16.5000]]],
        grad_fn=<AddBackward0>),
 tensor([[[ 16.7873,   3.9872,   0.5610,  ..., -13.7077,  11.3955,  18.0485]]],
        grad_fn=<AddBackward0>),
 tensor([[[ 22.2562,  11.6295,   3.8558,  ..., -24.7709,   5.1860,  14.1023]]],
        grad_fn=<AddBackward0>),
 tensor([[[27.6290, 22.7810,  2.6495,  ..., -4.8424,  5.9881, 12.5329]]],
        grad_fn=<AddBackward0>),
 tensor([[[28.2655,  2.1479, 21.5410,  ..., -8.8447, -5.6392,  0.8634]]],
        grad_fn=<AddBackward0>),
 tensor([[[ 27.9577, -18.5634,  16.0729,  ...,  -0.6811,  -9.6785,   1.6732]]],
        grad_fn=<AddBackward0>),
 tensor([[[34.2803, -9.0215,  3.6110,  ..., 12.6973, -8.0849,  3.3641]]],
        grad_fn=<AddBackward0>),
 tensor([[[  7.0729, -36.1400,   3.7265,  ...,  22.2282,   8.5002,   6.6390]]],
        grad_fn=<AddBackward0>),
 tensor([[[  2.7952, -49.4387,  -2.5552,  ...,  24.9252,   3.3260,  -4.5877]]],
        grad_fn=<AddBackward0>),
 

Keyword arguments {'add_special_tokens': False} not recognized.


{'input_ids': tensor([[256022,  68133,  53302,    983,    321,  22858,     70,    118,  83310,
         247681,  16782,   2909,     61,  23514,  47418,  73458,   5201, 215346,
         247736,   4118, 247717,   6128,   4015,  14490,     10, 229134,     70,
         161998,    290,    321,  48396, 247711,  42854,    525,    615, 247711,
           6297,   3868, 130434, 247673, 247676,   1078,  59264,  68798,   1518,
           3559,    488,   2783,   6662,   1613, 236671,  15333,    378, 219343,
         247676,  25634, 214120,  23514, 186284, 247813,  10201,  14260,  15327,
          43463,    665,  97999, 247780,   5070,   7979,   9452,   8192,  35369,
            321, 179626,  28093, 247711, 214867, 136970,    243,  94168,  74078,
          79056,  70685, 247676,      3,      0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1

['__eng__',
 'Insp',
 'ired',
 'by',
 'the',
 'success',
 'in',
 'N',
 'LP',
 ',',
 'emb',
 'odi',
 'ed',
 'agent',
 'research',
 '[2',
 '9,',
 '11,',
 '9',
 '4,',
 '2',
 '3]',
 'has',
 'seen',
 'a',
 'surge',
 'in',
 'adoption',
 'of',
 'the',
 'large',
 '-',
 'sc',
 'ale',
 'pre',
 '-',
 'tra',
 'ining',
 'paradig',
 'm',
 '.',
 'The',
 'recent',
 'advan',
 'ces',
 'can',
 'be',
 'ro',
 'ugh',
 'ly',
 'divided',
 'into',
 '4',
 'categories',
 '.',
 '1)',
 'Novel',
 'agent',
 'architecture',
 ':',
 'Dec',
 'ision',
 'Trans',
 'former',
 '[',
 '19,',
 '5',
 '8,',
 '14',
 '4]',
 'app',
 'lies',
 'the',
 'powerful',
 'self',
 '-',
 'attention',
 'models',
 'to',
 'sequ',
 'ential',
 'decision',
 'making',
 '.',
 '</s>',
 '<pad>']